In [40]:
import torch

path = "/home/tuyendv/Desktop/codes/ess_data_crawler_pipline/speaker_diarization/outputs/ckpts/MultiscaleDiarDecoder--val_loss=8.3349-epoch=9.ckpt"
state_dict_1 = torch.load(path, map_location="cpu")

In [41]:
path = "/home/tuyendv/Desktop/codes/ess_data_crawler_pipline/speaker_diarization/outputs/ckpts/MultiscaleDiarDecoder--val_loss=2.0436-epoch=3.ckpt"
state_dict_2 = torch.load(path, map_location="cpu")

In [45]:
new_state_dict = {}
for key, value in state_dict_2.items():
    if key != 'state_dict':
        new_state_dict[key] = value
    else:
        new_state_dict[key] = state_dict_1[key]

In [46]:
torch.save(new_state_dict, "/home/tuyendv/Desktop/codes/ess_data_crawler_pipline/speaker_diarization/outputs/ckpts/MSDD.ckpt")

In [42]:
state_dict_2

{'epoch': 3,
 'global_step': 29448,
 'pytorch-lightning_version': '1.8.0',
 'state_dict': OrderedDict([('preprocessor.featurizer.window',
               tensor([0.0000e+00, 6.1989e-05, 2.4796e-04, 5.5784e-04, 9.9158e-04, 1.5491e-03,
                       2.2301e-03, 3.0347e-03, 3.9624e-03, 5.0132e-03, 6.1867e-03, 7.4826e-03,
                       8.9007e-03, 1.0441e-02, 1.2102e-02, 1.3884e-02, 1.5787e-02, 1.7810e-02,
                       1.9952e-02, 2.2214e-02, 2.4594e-02, 2.7091e-02, 2.9706e-02, 3.2438e-02,
                       3.5286e-02, 3.8249e-02, 4.1326e-02, 4.4517e-02, 4.7821e-02, 5.1238e-02,
                       5.4765e-02, 5.8403e-02, 6.2150e-02, 6.6006e-02, 6.9970e-02, 7.4040e-02,
                       7.8216e-02, 8.2497e-02, 8.6881e-02, 9.1367e-02, 9.5955e-02, 1.0064e-01,
                       1.0543e-01, 1.1031e-01, 1.1530e-01, 1.2037e-01, 1.2554e-01, 1.3081e-01,
                       1.3616e-01, 1.4161e-01, 1.4714e-01, 1.5277e-01, 1.5847e-01, 1.6427e-01,
       

### filter

In [1]:
import os
import librosa
import soundfile as sf
from glob import glob
from tqdm import tqdm

In [15]:
input_path = "/home/tuyendv/Desktop/codes/ess_data_crawler_pipline/speaker_diarization/outputs/segments"
output_path = "/home/tuyendv/Desktop/codes/ess_data_crawler_pipline/outputs/speaker_diarization"

speakers = glob(f'{input_path}/speaker_*')
for speaker in tqdm(speakers):
    speaker_name = speaker.split("/")[-1]
    wavs = glob(f'{speaker}/*.wav')

    out_path = f'{output_path}/{speaker_name}'
    if not os.path.exists(out_path):
        os.mkdir(out_path)

    for wav_path in wavs:
        wav, sr = librosa.load(wav_path, sr=16000)

        if wav.shape[0]/sr < 1:
            continue
        wav_name = wav_path.split("/")[-1]
        wav_path = f'{out_path}/{wav_name}'
        sf.write(wav_path, wav, samplerate=sr)


100%|██████████| 5/5 [00:00<00:00,  6.64it/s]


### speaker verification

In [44]:
import os
import nemo
import re
import nemo.collections.asr as nemo_asr
from omegaconf import OmegaConf
from glob import glob
import random
from tqdm import tqdm
import shutil
import json
import torch
import torch
from torchmetrics.functional import pairwise_cosine_similarity
import shutil

In [17]:
# load data
path = "/home/tuyendv/Desktop/codes/ess_data_crawler_pipline/outputs/speaker_diarization"
files = glob(f'{path}/*/*.wav')

speaker2wavs = {}
for _file in files:
    path = _file
    temp = _file.split("/")[-1]
    wav_id = re.sub("_\d+\.wav","", temp)
    
    if wav_id not in speaker2wavs:
        speaker2wavs[wav_id] = [path,]
    else:
        speaker2wavs[wav_id].append(path)

In [45]:
MODEL_CONFIG = 'config/titanet-large.yaml'
config = OmegaConf.load(MODEL_CONFIG)

verification_model = nemo_asr.models.EncDecSpeakerLabelModel.load_from_checkpoint(
    "outputs/ckpts/TitaNet-L--val_loss=1.0419-epoch=45.ckpt",
    map_location="cpu",
    hparams_file="config/hparams.yml")

verification_model.eval()

[NeMo W 2023-05-17 16:28:32 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: datas/train.json
    sample_rate: 16000
    labels:
    - spk_0
    - spk_1
    - spk_10
    - spk_100
    - spk_101
    - spk_102
    - spk_103
    - spk_104
    - spk_105
    - spk_106
    - spk_107
    - spk_108
    - spk_109
    - spk_11
    - spk_110
    - spk_111
    - spk_112
    - spk_113
    - spk_114
    - spk_115
    - spk_116
    - spk_117
    - spk_118
    - spk_119
    - spk_12
    - spk_120
    - spk_121
    - spk_122
    - spk_123
    - spk_124
    - spk_125
    - spk_126
    - spk_127
    - spk_128
    - spk_129
    - spk_13
    - spk_130
    - spk_131
    - spk_132
    - spk_133
    - spk_134
    - spk_135
    - spk_136
    - spk_137
    - spk_138
    - spk_139
    - spk_14
    - spk_140
    - spk_141
    - spk_142
    

[NeMo I 2023-05-17 16:28:32 features:291] PADDING: 16


EncDecSpeakerLabelModel(
  (loss): AngularSoftmaxLoss()
  (eval_loss): AngularSoftmaxLoss()
  (_accuracy): TopKClassificationAccuracy()
  (preprocessor): AudioToMelSpectrogramPreprocessor(
    (featurizer): FilterbankFeatures()
  )
  (encoder): ConvASREncoder(
    (encoder): Sequential(
      (0): JasperBlock(
        (mconv): ModuleList(
          (0): MaskedConv1d(
            (conv): Conv1d(80, 80, kernel_size=(3,), stride=(1,), padding=(1,), groups=80, bias=False)
          )
          (1): MaskedConv1d(
            (conv): Conv1d(80, 1024, kernel_size=(1,), stride=(1,), bias=False)
          )
          (2): BatchNorm1d(1024, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (3): SqueezeExcite(
            (fc): Sequential(
              (0): Linear(in_features=1024, out_features=128, bias=False)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=128, out_features=1024, bias=False)
            )
            (gap): AdaptiveAvgPool1d(

In [46]:
def infer_verification(paths_1, paths_2, threshold=0.7):
    embedding_1, embedding_2 = [], []
    for path_1 in paths_1:
        embedding = verification_model.get_embedding(path_1)
        embedding_1.append(embedding)
    
    for path_2 in paths_2:
        embedding = verification_model.get_embedding(path_2)
        embedding_2.append(embedding)

    embedding_1 = torch.cat(embedding_1, dim=0)
    embedding_2 = torch.cat(embedding_2, dim=0)

    cos_sim = pairwise_cosine_similarity(embedding_1, embedding_2)
    similarity_score = (cos_sim + 1) / 2

    positive = (similarity_score > threshold).sum()
    negative = (similarity_score < threshold).sum()

    if positive > 8:
        return True
    return False


In [47]:
paths_1 = [
    "/home/tuyendv/Desktop/codes/ess_data_crawler_pipline/outputs/speaker_diarization/speaker_0/wav_0.rttm_speaker_0_0.wav",
    "/home/tuyendv/Desktop/codes/ess_data_crawler_pipline/outputs/speaker_diarization/speaker_0/wav_0.rttm_speaker_0_16.wav",
    "/home/tuyendv/Desktop/codes/ess_data_crawler_pipline/outputs/speaker_diarization/speaker_0/wav_0.rttm_speaker_0_18.wav"]
paths_2 = [
    "/home/tuyendv/Desktop/codes/ess_data_crawler_pipline/outputs/speaker_diarization/speaker_0/wav_0.rttm_speaker_0_19.wav",
    "/home/tuyendv/Desktop/codes/ess_data_crawler_pipline/outputs/speaker_diarization/speaker_0/wav_0.rttm_speaker_0_20.wav",
    "/home/tuyendv/Desktop/codes/ess_data_crawler_pipline/outputs/speaker_diarization/speaker_0/wav_0.rttm_speaker_0_21.wav"   
]
infer_verification(paths_1, paths_2)

False

In [48]:
speaker_map = {}

for speaker_1, wavs_1 in tqdm(speaker2wavs.items()):
    if len(wavs_1) > 8:
        wavs_1 = random.sample(wavs_1, 8)
    for speaker_2, wavs_2 in speaker2wavs.items():
        if speaker_1 == speaker_2:
            continue
        if speaker_1 in speaker_map and speaker_2 in speaker_map:
            continue
        if len(wavs_2) > 8:
            wavs_2 = random.sample(wavs_2, 8)
        res = infer_verification(wavs_1, wavs_2)

        if res == True:
            if speaker_1 not in speaker_map:
                speaker_map[speaker_1] = [speaker_2,]
            else:
                speaker_map[speaker_1].append(speaker_2)

            if speaker_2 not in speaker_map:
                speaker_map[speaker_2] = [speaker_1,]
            else:
                speaker_map[speaker_2].append(speaker_1)

100%|██████████| 10/10 [02:54<00:00, 17.49s/it]


In [49]:
for key, value in speaker2wavs.items():
    if key not in speaker_map:
        speaker_map[key] = [key,]

In [50]:
speaker2id = {}
count = 0
for key, value in speaker_map.items():
    if key not in speaker2id:
        speaker2id[key] = f"speaker_{count}"
        count += 1
        for mapped_spk in value:
            speaker2id[mapped_spk] = speaker2id[key]


In [51]:
path = "/home/tuyendv/Desktop/codes/ess_data_crawler_pipline/outputs/speaker_diarization/*/*.wav"
output_path = "/home/tuyendv/Desktop/codes/ess_data_crawler_pipline/outputs/datas"
os.system(f"rm -r {output_path}/*")

files = glob(path)
speaker2num_utt = dict.fromkeys(speaker2id.keys(), 0)

for _file in tqdm(files):
    _id = _file.split("/")[-1]
    _id = "_".join(_id.split("_")[:-1])
    output_dir = f'{output_path}/{speaker2id[_id]}'
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    # wav, sr = librosa.load(_file, sr=16000)
    shutil.copy(_file, f'{output_dir}/{_file.split("/")[-1]}')
    # sf.write(f'{output_dir}/{speaker2num_utt[_id]}.wav', wav, samplerate=sr)
    speaker2num_utt[_id] += 1

100%|██████████| 153/153 [00:00<00:00, 5459.24it/s]
